In [1]:
import ipywidgets as widgets

# Widgets

In [2]:
# This function creates a handful of widgets that
# allow users to select the phenotype of a bird
# If we then assume all traits breed true, we
# can generate a genotype without making the
# user understand bird genetics
def createPeacockPhenotypeWidgets(peacockName, sex):
    colorList = ['Wild Type'] + list(set(
        # TECH EXPLANATION
        # lambda indicates an inline function
        # map() means that the lambda function is called for each item in the colorGenes array
        # It converts each gene's dictionary into just the name column
        # list() turns that map back into an array
        # The goal here is to make sure all genes show up as human readable options 
        # in the list, and they don't get out of sync with each other
        list(map(lambda gene: gene['name'], colorGenes)) + 
        list(map(lambda allotype: allotype['name'], sexLinkedColorAllotypes)) +
        list(map(lambda phenotype: phenotype['name'], multiGeneColors)) +
        list(map(lambda phenotype: phenotype['name'], sexAndAutosomalComboColors)) +
        # Only add the het sex colors if the bird is male - female birds can't have het sex traits
        (list(map(lambda phenotype: phenotype['name'], hetSexColors)) if sex == 'Male' else [])))
        
    
    color = widgets.Dropdown(
        options=colorList,
        value=colorList[0],
        description='Color:',
        disabled=False,
    )

    # See colorList for technical explanation
    patternList = ['Barred Wing Wild Type'] + list(set(
        list(map(lambda gene: gene['name'], patternGenes))))
    pattern = widgets.Dropdown(
        options=patternList,
        value=patternList[0],
        description='Pattern:',
        disabled=False,
    )
    
    # See colorList for technical explanation
    piedList = ['Non-Leucistic Wild Type'] + list(set(
       list(map(lambda allotype: allotype['name'], piedAllotypes)) +
       list(map(lambda phenotype: phenotype['name'], hetPied))))
    
    pied = widgets.Dropdown(
        options=piedList,
        value=piedList[0],
        description='Pied:',
        disabled=False,
    )

    eyeList = ['Non-Leucistic Wild Type'] + list(set(
       list(map(lambda allotype: allotype['name'], whiteEyeAllotypes))))
    
    eye = widgets.Dropdown(
        options=eyeList,
        value=eyeList[0],
        description='Leucistic Eye:',
        disabled=False,
    )

    return (
        widgets.VBox([
            widgets.Label(value=peacockName),
            color, pattern, pied, eye
        ]),
        {
            'color': color,
            'pattern': pattern,
            'pied': pied,
            'eye': eye
        }
    )

In [3]:
# This lets us turn a single gene into a genotype widget
# We will run this function over all the autosomal color genes
# and the shoulder genes to generate appropriate widgets
def createGenotypeWidgetFromGene(gene):
    possibilities = [
        'WT/WT', 
        'WT/' + gene['notation'],
        gene['notation'] + '/' + gene['notation']
    ]
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description=gene['name'] + ':',
        disabled=False,
    )

# For sex-linked genes, we want to change the genotype based on sex,
# and treat different allotypes as mutually exclusive
def createMultiAllotypeGeneWidget(geneName, wildType, allotypes, sex, geneIsSexLinked = False):
    allotypesWithWild = [wildType] + allotypes
    possibilities = []

    # Iterate through each allotype as a possible first allele.
    # In this case, we use enumerate instead of a normal for each loop so we can limit
    # the loop of the second allele and prevent duplicate genotypes
    for index, firstAllele in enumerate(allotypesWithWild):
        if geneIsSexLinked and sex == 'Female':
            # For females just add the missing sex chromasome 'w' to the first allele
            possibilities.append(firstAllele['notation'] + '/w');
        else:
            # For double-allele genes, iterate through each allotype starting with the firstAllele as a possible second allele
            # This gives us every possible combination of alleles without duplicates 
            # You can think of it as the top trianlge of a possibilities square
            # The [index:] selects all items in the list starting with the same item as the current firstAllele
            for secondAllele in allotypesWithWild[index:]:
                possibilities.append(firstAllele['notation'] + '/' + secondAllele['notation'])
    
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description=geneName+ ":",
        disabled=False,
    )

# Used for troubleshooting bugs with print()
output = widgets.Output()

# When genotype changes, update the related phenotype
# TODO - add input arrays to saveGenotypeToBird
def handleGenotypeChange(genotypeWidgets, phenotypeWidget, bird, sex, geneName, newGenotype, wildType, simpleGenes, multiAllotypeGenes, 
                         hetTraits, multiGeneTraits):
    with output:
        saveGenotypeToBird(bird, geneName, newGenotype)
        (phenotype, prevPhenotypes, unknown) = getPhenotypeFromBird(bird, sex, wildType, simpleGenes, multiAllotypeGenes, 
                                                                    hetTraits, multiGeneTraits)
        
        # We don't want any genotype widgets to be altered by changes to the phenotype widget
        for widget in genotypeWidgets:
            widget.unobserve(None, 'value')
        
        # Set the index and the value of the phenotype widget to match the new genotype
        # We have to set both, or else the display won't update properly
        phenotypeWidget.index = phenotypeWidget.options.index(phenotype)
        phenotypeWidget.value = phenotype
        
        # Once all changes are made, we need to re-build the observers
        for widget in genotypeWidgets:
            widget.observe(lambda changedValue: 
                               handleGenotypeChange(
                                   genotypeWidgets, phenotypeWidget, bird, sex, geneName, changedValue['new'], 
                                   wildType, simpleGenes, multiAllotypeGenes, hetTraits, multiGeneTraits), 
                               'value')

def handlePhenotypeChange(thisWidget, genotypeWidgets, bird, sex, phenotype, simpleGenes,
                          multiAllotypeGenes, sexLinkedAllotypes, multiGeneTraits, sexAndAutosomalCombos, hetSexTraits):
    with output:
        savePhenotypeToBird(bird, phenotype, sex, simpleGenes, multiAllotypeGenes, sexLinkedAllotypes, 
                            multiGeneTraits, sexAndAutosomalCombos, hetSexTraits)
        # We don't want the phenotype widget to be altered by half-finished changes, so we
        # need to temorarily prevent the phenotype from changing when the genotype changes
        thisWidget.unobserve(None, 'value')

        # We may need to change multiple genotype widgets in order to match the 
        # current phenotype
        for widget in genotypeWidgets:
            if widget.description[:-1] in bird:
                widget.value = bird[widget.description[:-1]]
                widget.index = widget.options.index(bird[widget.description[:-1]])
            else:
                widget.value = widget.options[0]
                widget.index = 0

        # Once all changes are made, we need to re-build the observer
        thisWidget.observe(lambda changedValue: 
                           handlePhenotypeChange(
                               thisWidget, genotypeWidgets, bird, sex, changedValue['new'], 
                               simpleGenes, multiAllotypeGenes, sexLinkedAllotypes, multiGeneTraits, 
                               [], hetSexTraits), 
                           'value')

def createPeacockGenotypeWidgets(peacockName, sex, phenotypeWidgets):
    bird = {}
    
    # For each possible color, create a widget with genotype options
    colors = []
    for gene in colorGenes:
        widget = createGenotypeWidgetFromGene(gene)
        colors.append(widget)
        # This updates the color phenotype whenever any color genotype changes
        widget.observe(
            lambda changedValue, geneName=gene['name']: 
                handleGenotypeChange(colors, phenotypeWidgets['color'], bird, sex, geneName, changedValue['new'], 
                                     "Wild Type", colorGenes, 
                                     [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }],
                                     hetSexColors, multiGeneColors), 
            'value')

    # Create a single widget for all the sex-linked color varients
    sexLinkedColor = createMultiAllotypeGeneWidget('Sex-Linked Color', { 'notation': 'Z(WT)', 'name': 'Wild Type' }, 
                                                   sexLinkedColorAllotypes, sex, True)
    colors.append(sexLinkedColor)
    # This updates the phenotype whenever the sex-linked color genotype changes
    sexLinkedColor.observe(
        lambda changedValue, sex=sex: 
            handleGenotypeChange(colors, phenotypeWidgets['color'], bird, sex, 'Sex-Linked Color', changedValue['new'],
                                 "Wild Type", colorGenes, 
                                 [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }],
                                 hetSexColors, multiGeneColors), 
        'value')

    # This updates the genotype(s) when the color phenotype changes
    phenotypeWidgets['color'].observe(
        lambda changedValue: 
            handlePhenotypeChange(phenotypeWidgets['color'], colors, bird, sex, changedValue['new'], 
                                  colorGenes, 
                                  [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], 
                                  sexLinkedColorAllotypes, multiGeneColors,  sexAndAutosomalComboColors, hetSexColors), 
            'value')

    # Create a single widget for Pied genes
    pied = createMultiAllotypeGeneWidget('Pied', { 'notation': 'WT', 'name': 'Non-Leucistic Wild Type' }, 
                                                   piedAllotypes, sex)
    # This updates the phenotype whenever the sex-linked color genotype changes
    pied.observe(
        lambda changedValue, sex=sex: 
            handleGenotypeChange(pied, phenotypeWidgets['pied'], bird, sex, 'Pied', changedValue['new'], 
                                "Non-Leucistic Wild Type", [], 
                                [{ 'name': 'Pied', 'allotypes': piedAllotypes, 'sexLinked': False }],
                                hetPied, []), 
        'value')

    # This updates the genotype(s) when the color phenotype changes
    phenotypeWidgets['pied'].observe(
        lambda changedValue: 
            handlePhenotypeChange(phenotypeWidgets['pied'], [pied], bird, sex, changedValue['new'], 
                                       [], 
                                       [{ 'name': 'Pied', 'allotypes': piedAllotypes, 'sexLinked': False }], 
                                       [], [],  [], hetPied), 
            'value')

    return (
        widgets.VBox([widgets.Label(value=peacockName)] + colors + [pied]),
        {
            'colors': colors,
            'pied': pied
        }
    )

In [4]:
def generatePeafowlImage(sex, color):
    # Color should be the base layer
    colorImageName = "Images/" + sex + "/Color/" + color + ".png"
    if not os.path.isfile(colorImageName):
        # Default to the unknown color if we don't have an image
        # for the defined color
        colorImageName = "Images/" + sex + "/Color/Unknown.png"

    # Open the file for the color image
    peafowlColor = Image.open(colorImageName)
    # Open the lineart image
    peafowlLineart = Image.open("Images/" + sex + "/lineart.png")

    # Merge the images as layers
    peafowlColor.paste(peafowlLineart, (0, 0), peafowlLineart)

    # Extract the image data as bytes
    imageData = io.BytesIO()
    peafowlColor.save(imageData, format='PNG')

    # Return a widget-compatible image
    return imageData